In [3]:
import requests as req 
from bs4 import BeautifulSoup as BS

from connection.var import ConnectionVar, agent_desktop, parser, URL

In [30]:
header = {'User-Agent': agent_desktop}
from datetime import datetime

In [5]:
def get_html(url, header):
    #get data from sources
    #url for various sources
    connect = req.get(url, header)
    data = connect.content
    return data

def parse_html(data):
    prettify = BS()

def save_to_db(data):
    #save data from parsed_html
    #maybe use CSV at the beginning 
    pass

In [6]:
tekst = get_html(URL[1], header=agent_desktop)

In [7]:
soup = BS(tekst, parser=parser)

In [10]:
print(f"Name: {name_tag:.>{30}} \nPrice: {price_tag:.>{32}} \nPrice Cap: {pricecap_tag:.>{34}} \nVolume: {volume24_tag:.>{36}} \nCirculation: {circulating_tag:.>{30}} \nPercent Changed:{percent_tag:.>{18}}")

Name: .......................Bitcoin 
Price: ......................$11,203.70 
Price Cap: ..................$206,723,072,734 
Volume: .....................$23,668,389,154 
Circulation: ................18,451,318 BTC 
Percent Changed:............-1.11%


In [11]:
all_name = [x.get_text() for x in soup.find_all('td', class_=ConnectionVar['NAME'])]
all_price = [x.get_text() for x in soup.find_all('td',class_=ConnectionVar['PRICE'])]
all_pricecap = [x.get_text() for x in soup.find_all('td', class_=ConnectionVar['PRICE'])]
all_volume24 = [x.get_text() for x in soup.find_all('td', class_=ConnectionVar['VOLUME_24'])]
all_circulation = [x.get_text() for x in soup.find_all('td', class_=ConnectionVar['CIRCULATION'])]
all_percent = [x.get_text() for x in soup.find_all('td', class_=ConnectionVar['PERCENT_CHG'])]

In [16]:
import pandas as pd

In [22]:
df = pd.DataFrame(index=all_name, columns=['PRICE','PRICE_CAP', 'VOLUME24','CIRCULATION', 'PERCENT_chg'])

In [27]:
df['PRICE'] = all_price
df['PRICE_CAP'] = all_pricecap
df['VOLUME24'] = all_volume24
df['CIRCULATION'] = all_circulation
df['PERCENT_chg'] = all_percent

In [52]:
with open(f"saved/{datetime.strftime(datetime.now(), '%m-%d-%Y')}_CRYPTO.csv",'w+') as f:
    f.write(df.to_csv(index=True))

In [58]:
new = pd.read_csv('saved/08-04-2020_CRYPTO.csv', index_col='Unnamed: 0')

In [59]:
new.head()

,PRICE,PRICE_CAP,VOLUME24,CIRCULATION,PERCENT_chg
Bitcoin,"$11,189.62","$11,189.62","$22,342,225,353","18,451,356 BTC",-1.72%
Ethereum,$387.77,$387.77,"$15,006,562,714","112,042,281 ETH",-1.39%
XRP,$0.299489,$0.299489,"$3,035,524,213","44,862,646,997 XRP *",-4.26%
Tether,$1.00,$1.00,"$34,595,154,082","9,998,221,723 USDT *",0.08%
Bitcoin Cash,$287.10,$287.10,"$2,063,254,152","18,480,675 BCH",-3.60%
